# QAOA
VQEアルゴリズムとほぼ同じですが、利用する量子回路であるAnsatzにQAOA専用の組合せ最適化問題向けのものを利用します。

## 今回学ぶこと
1. QAOAの仕組み
2. 簡単な例題でQAOAの実装をする

## Blueqatのインストール
pipからBlueqatをインストールします。今回は追加でObaqもインストールします。

In [1]:
!pip install blueqat obaq

----

## 量子断熱計算
QAOAは時間発展の量子断熱計算と呼ばれる仕組みを使います。   
現時刻をt、全体のスケジュールをTとし、ハミルトニアンは二つを使います。準備する初期のハミルトニアン $H_{start}$とし、解きたい問題を入れたハミルトニアンを $H_{final}$ とします。その時に、下記のように初期のハミルトニアンから解きたい問題のハミルトニアンへと少しずつ入れ替えていきます。

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{final}
$$


$T\rightarrow\infty$ のとき、基底状態は以下のような固有状態になります。

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

この2つのハミルトニアンを入れ替える過程を時間発展演算子を利用して作ります。

$$
\mid \psi_{t+1} \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

## 量子回路
量子回路は、

1. 初期状態の準備
2. QAOA Ansatz

から構成されます。

```
|0> ---[初期状態の準備]---[QAOA Ansatz]---[測定]
```

より詳しく中身を見ると、具体的な回路例を使って、

```
|0> --H--  --RZZ--RZ--RX- -RZZ--RZ--RX--  --[測定]
|0> --H--  --*----RZ--RX- -*----RZ--RX--  --[測定]
```

このようになっており、一番左が初期状態の準備、次がQAOA Ansatzの1ステップ目、その次がQAOA Ansatzの2ステップ目です。最後に測定をして解を求めます。

## 2つのハミルトニアン
今回は初期のハミルトニアン（ミキサーハミルトニアン）と、解きたい問題のハミルトニアン（コストハミルトニアン）を使います。ミキサーハミルトニアンは問題の制約条件によって選び方を変えます。コストハミルトニアンは問題において最小化したい値を定式化します。

## ミキサーハミルトニアンと初期状態の準備
ミキサーハミルトニアンは問題の探索の仕方を決めるためのものです。量子アニーリングでは、Xをミキサーハミルトニアンとして、量子ビットの値を反転させながら答えを探索します。

ミキサーハミルトニアンを問題の制約条件によって決めると、初期状態が決まります。初期状態の準備は、ミキサーハミルトニアンの固有状態を設定します。上記XをミキサーハミルトニアンとしたときのXミキサーを利用する場合、初期状態に設定する固有状態$\mid \psi \rangle$は$\mid + \rangle$などを選びます。



## コストハミルトニアン
コストハミルトニアンは問題によって定式化を変えます。社会問題をベースとして組合せ最適化問題と呼ばれる種類の問題を、物理のイジングモデルと呼ばれるモデルの上に落とし込みます。コストハミルトニアンはZ演算子の期待値が+1と-1であることを利用して、問題の条件を全てZもしくはZZ演算子を使って表現します。

```
hamiltonian = -Z(0) - Z(0)*Z(1)
```

のように、単体のZの前に「バイアス」を設定し、複数のZの前に「ウェイト」を設定します。

## QAOA Ansatzとハミルトニアンの時間発展
QAOA Ansatzの中には問題設定をしたコストハミルトニアンと探索の仕方を決めるミキサーハミルトニアンが時間発展演算子と呼ばれる形に変形され格納されています。

$X$ゲートの時間発展は$RX(\theta)$ゲートのように角度を含むゲートを利用します。また、2量子ビットは$XX$ゲートの時間発展も、$RXX(\theta)$のように角度を含むものになります。

## ステップ数
量子断熱計算は通常アナログの過程を時間発展演算子を利用して離散化しています。より精度を上げるためには、分割数を増やすことで精度を上げられますが、回路が長くなるので、その分計算時間がかかります。QAOAでもステップ数を設定することで精度を上げることができます。ステップ数はQAOA Ansatzの中の繰り返しの回数に対応しています。

## Xミキサー
代表的なミキサーにXミキサーがあります。

$$
X\mid 0 \rangle = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
1\\
0
\end{bmatrix}
=
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

これは量子ビットの値を反転させるため、単体の量子ビットに作用させて使います。初期状態も単体の量子ビットにHゲートを作用させて$\mid + \rangle$状態を作ります。

## XYミキサー
XYミキサーは二つの状態を入れ替えながら探索を行います。(XX+YY)/2は、

$$
X_0X_1 + Y_0Y_1 = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
+
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
=
\begin{bmatrix}
0&0&0&1\\
0&0&1&0\\
0&1&0&0\\
1&0&0&0
\end{bmatrix}
+
\begin{bmatrix}
0&0&0&-1\\
0&0&1&0\\
0&1&0&0\\
-1&0&0&0
\end{bmatrix}
$$

ですので、足し合わせて2で割ると、

$$
(X_0X_1 + Y_0Y_1)/2 
=
\begin{bmatrix}
0&0&0&0\\
0&0&1&0\\
0&1&0&0\\
0&0&0&0
\end{bmatrix}
$$

となりました。こちらは、01と10状態を交換しながら探索を行います。初期状態は固有状態として、01と10のもつれ状態を選びます。また、

$$
(X_0X_1 - Y_0Y_1)/2 = 
\begin{bmatrix}
0&0&0&1\\
0&0&0&0\\
0&0&0&0\\
1&0&0&0
\end{bmatrix}
$$

となり、こちらは00と11の状態を交換します。初期状態は00と11のもつれ状態を選びます。

## 例題：量子アニーリング

ObaqにQaoaQaoaAnsatzを入れました。QaoaQaoaAnsatzは初期状態では量子アニーリングに設定されています。mixerにXというものを使い、初期状態に|+>を設定しています。初期状態の|+>は全ての量子ビットにアダマールゲートHをかけることで実現します。

```
|0> --H--  --RZZ--RZ--RX- -RZZ--RZ--RX--  --[測定]
|0> --H--  --*--------RX- -*--------RX--  --[測定]
```

In [8]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import Vqe
from obaq.ansatz import *

#mixerの準備
Xmixer = X[0]+X[1]
Xinit = Circuit(2).h[0,1]

h = -Z(0) -Z(0)*Z(1)
step = 2

result = Vqe(QaoaQaoaAnsatz(h, step, Xinit, Xmixer)).run()
print(result.most_common(12))

(((0, 0), 0.5248588835345177), ((1, 1), 0.2273829345062457), ((0, 1), 0.2126896364367128), ((1, 0), 0.035068545522521914))


また、初期状態とミキサーを指定しない場合は量子アニーリングが採用されます。

In [10]:
result = Vqe(QaoaQaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((0, 0), 0.5520992030571237), ((1, 1), 0.28865065110869187), ((0, 1), 0.1506319265155627), ((1, 0), 0.008618219318620222))


## 例題：XYミキサー
次に同じ問題でXYミキサーを使ってみます。初期状態にはもつれ状態を指定します。ミキサーには(XX+YY)/2のハミルトニアンを渡します。

In [11]:
#mixerの準備
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1]
XYinit = Circuit().h[0].cx[0,1].x[0]

result = Vqe(QaoaQaoaAnsatz(h, step, XYinit, XYmixer)).run()
print(result.most_common(12))

(((0, 1), 0.9999999999999984), ((1, 0), 1.5018775220873225e-18), ((1, 1), 3.466673890819865e-33))


となりました。本来は(0,0)が出て欲しいところですが、01と10で制約をかけましたので、01と10の答えの中から正解に近いものを探そうとしています。